# PM2.5 Forecasting

In [ ]:
import optuna # Hyperparameter optimization
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.optimizers import Adam

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

## Search hyperparameters

In [ ]:
class MLPOptimizer:
  """
  Class for hyperparameter optimizing for PM2.5 Forecaster.
  """
  def __init__(self, input_dim:int, n_trials:int):
    self.input_dim = input_dim
    self.n_trials = n_trials
    self.study = None
    self.X_train = None
    self.y_train = None
    self.X_val = None
    self.y_val = None

  def create_model(self, trial):
    model = Sequential()

    # Hyperparameter selection
    neurons = trial.suggest_categorical("neurons", [8, 12, 16, 24, 32, 40])
    neurons2 = trial.suggest_categorical("neurons2", [8, 12, 16, 24, 32, 40])
    dropout_rate = trial.suggest_float("dropout", 0.1, 0.2, log=True)
    # reg_type = trial.suggest_categorical("reg_type", [None])
    # reg_strength = trial.suggest_float("reg_strength", 1e-3, 1e-1, log=True)

    # Choose regularization
    #if reg_type == "l1":
     # reg = l1(reg_strength)
    #elif reg_type == "l2":
     # reg = l2(reg_strength)
    #else:
     # reg = None

    # Hidden layer
    model.add(Dense(
        neurons,
        activation="relu",
        input_dim=self.input_dim
        ))

    # Dropout layer
    model.add(Dropout(dropout_rate))

    # Output layer
    model.add(Dense(1))

    # Compile model
    learning_rate = trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True)
    model.compile(optimizer=Adam(learning_rate), loss="mse")

    return model

  def objective(self, trial):
    model = self.create_model(trial)
    epochs = trial.suggest_categorical("epochs", [50, 100])

    # Train the model
    model.fit(
        self.X_train,
        self.y_train,
        epochs=epochs,
        validation_data=(self.X_val, self.y_val),
        verbose=0
        )

    # Evaluate on validation data
    loss = model.evaluate(self.X_val, self.y_val, verbose=0)

    return loss

  def optimize(self, X_train:np.array, y_train:np.array, X_val, y_val:np.array):
    self.X_train = X_train
    self.y_train = y_train
    self.X_val = X_val
    self.y_val = y_val

    self.study = optuna.create_study(direction="minimize")
    self.study.optimize(self.objective, n_trials = self.n_trials)

    print("Best hyperparameters:", self.study.best_params)
    return self.study

In [ ]:
mlp_optimizer = MLPOptimizer(n_steps, 32)
study = mlp_optimizer.optimize(X_train, y_train, X_val, y_val)